<h1 style="color:#000;font-size:30px;background-color:#15C6B4;width:100%;height:30px;text-align:center">Weekday-Text Classification</h1>

<p>
<h4 style="color:#ff6871;">Last edited:-<span style="color:#808080"> Pranav Purankar 28-Oct 2023</span></h4>
<h4 style="color:#ff6871;">Team:-<span style="color:#808080"> Product Management</span></h4>
<h4 style="color:#ff6871;">Details:-<span style="color:#808080;line-height:18pt"> This file contains a pre-processed data. This data can be useful for classifying a particular candidate; to indetify which industry segment he/she/they belongs to. Kindly refer this GitBook to get an in-depth overview of the project:- <a>URL(http:www.google.com)</a></span></h4>
</p>

In [1]:
# Required libraries and modules

import pandas as pd
import numpy as np
import string
import re
from ydata_profiling import ProfileReport
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# import os
# import spacy
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.linear_model import SGDClassifier

In [2]:
df=pd.read_csv('Enrich_Company_Final.csv')

In [3]:
# Summary of the existing data
prof=ProfileReport(df)
prof.to_file(output_file='company_data_summary.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
df.sample(frac=1)

,Description,Website,Slug,Country,Enrich Company,Employee Count,Company URL
620,autonomous checkout technology platform retail...,brysk.com,brysk,US,Brysk,17,https://linkedin.com/company/niflr\r
477,kognitiv redefines loyalty helping brands secu...,kognitiv.com,kognitiv-corporation,CA,Kognitiv Corporation,355,http://www.linkedin.com/company/kognitiv-corpo...
399,hono began 2008 pioneering hrtech solutions em...,hono.ai,connectwithhono,IN,HONO,219,http://linkedin.com/company/connectwithhono\r
573,mool established simple purpose advise worlds ...,mool.one,moolmantra,IN,Mool,28,http://linkedin.com/company/moolmantra/\r
496,missing,leap.club,leapclub,IN,leap.club,150,https://linkedin.com/company/leapclub\r
...,...,...,...,...,...,...,...
79,axessai product hardmate inc delaware c corpor...,axess.ai,axess-ai,US,axessAI,1,https://www.linkedin.com/company/axess-ai/
360,gromo mission increase accessibility financial...,gromo.in,gromofintech,IN,GroMo,467,https://linkedin.com/company/gromofintech/about\r
128,cbazaar among earliest ecommerce companies ind...,cbazaar.com,cbazaar,IN,Cbazaar.com,240,http://linkedin.com/company/cbazaar
760,refrens place independant freelancers b2b serv...,refrens.com,refrens,IN,Refrens,38,https://linkedin.com/company/refrens\r


In [4]:
#Combining columns, filling Na values with relevant data, dropping unwanted columns

In [5]:
df.drop(columns=['Org Id','Created At','Size','Locality','Updated At'], inplace=True)

In [6]:
df['Employee Count']=df['Employee Count'].fillna(0).apply(np.int32)

In [7]:
df['New_Industry_filled_by_specialities']=df['Industry'].fillna(df['Specialities'])

In [8]:
df['concatenate']=df['New_Industry_filled_by_specialities'] + ", " + df['Specialities']

In [9]:
df['concatenate']=df['concatenate'].fillna(df['New_Industry_filled_by_specialities'])

In [10]:
df.rename(columns={'concatenate':'Industry_Specialities'},inplace=True)

In [11]:
df['Description']=df['Description'] + " " + df["Industry_Specialities"]

In [12]:
df.drop(columns=['Industry','Specialities','New_Industry_filled_by_specialities','Industry_Specialities'],inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Description     689 non-null    object
 1   Website         765 non-null    object
 2   Slug            782 non-null    object
 3   Country         782 non-null    object
 4   Enrich Company  782 non-null    object
 5   Employee Count  783 non-null    int32 
 6   Company URL     782 non-null    object
dtypes: int32(1), object(6)
memory usage: 39.9+ KB


In [14]:
'''
Storing NaN values in the description, every bit of information is essential.
This is vital column for our whole analysis process.
'''
# nan_values = df[df['Description'].isna()]
# df=pd.DataFrame(nan_values)
# df.to_csv('nan_values.csv',index=True)


'\nStoring NaN values in the description, every bit of information is essential.\nThis is vital column for our whole analysis process.\n'

In [15]:
df['Description']=df['Description'].fillna('Missing')
df['Website']=df['Website'].fillna('nourl')

In [16]:
df=df.dropna()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 782 entries, 0 to 781
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Description     782 non-null    object
 1   Website         782 non-null    object
 2   Slug            782 non-null    object
 3   Country         782 non-null    object
 4   Enrich Company  782 non-null    object
 5   Employee Count  782 non-null    int32 
 6   Company URL     782 non-null    object
dtypes: int32(1), object(6)
memory usage: 45.8+ KB


<h2 style="color:#ffb700"> Text Pre-processing </h2>
<p style="color:#808080;font-size:20px"> 1. Lowercasing letters</p>

In [18]:
df['Description']=df['Description'].str.lower()
df['Description'].sample(frac=1)

/tmp/ipykernel_26545/4094036355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description']=df['Description'].str.lower()


237    an indo-norwegian cloud-software company on a ...
335    getfocus (focus analytics pvt. ltd.) is an exc...
495                                              missing
254    checkout financing, lending and payments platf...
290    filo is the worlds only live instant tutoring ...
                             ...                        
361    hey!\n\ngrow indigo was established in 2018 by...
373    haikujam is a mobile game that promotes self-c...
183    credihealth is an online solution to all your ...
419    a leading future tech-based professional educa...
202    dass scientific research labs private limited ...
Name: Description, Length: 782, dtype: object

<p style="color:#808080;font-size:20px"> 2. Removing Punctuation</p>

In [19]:
def remove_punctuations(text):
    punctuations=string.punctuation
    return text.translate(str.maketrans('','',punctuations))

In [20]:
df['Description']=df['Description'].apply(lambda x: remove_punctuations(x))

/tmp/ipykernel_26545/199114757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description']=df['Description'].apply(lambda x: remove_punctuations(x))


<p style="color:#808080;font-size:20px"> 3. Removing Stopwords</p>

In [21]:
# " ".join(stopwords.words('english'))

In [22]:
STOPWORDS=set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

In [23]:
df['Description']=df['Description'].apply(lambda x: remove_stopwords(x))

/tmp/ipykernel_26545/3129844413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description']=df['Description'].apply(lambda x: remove_stopwords(x))


<p style="color:#808080;font-size:20px"> 4. Removing Special Characters</p>

In [24]:
def remove_spl_chars(text):
    text=re.sub('[^a-zA-Z0-9]'," ",text)
    text=re.sub('\s+',' ',text)
    return text

In [25]:
df['Description']=df['Description'].apply(lambda x: remove_spl_chars(x))

/tmp/ipykernel_26545/2318416937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description']=df['Description'].apply(lambda x: remove_spl_chars(x))


<h2 style="color:#ffb700"> Data Annotation </h2>
<p style="color:#808080;font-size:20px"> 1. Each row as a text file</p>

In [26]:
d = df['Description']
file = 'file{}.txt'

n = 0 # to number the files
for row in d.items():
    with open(file.format(n), 'w') as f:
        f.write(str(row))
        n += 1

<p style="color:#808080;font-size:20px"> 2. Labelled data (doccano output)</p>

In [27]:
df_json=pd.read_json('all.jsonl',lines=True)

ValueError: Expected object or value

In [ ]:
# df_json['label']

<h2 style="color:red"> Why do you stop here? </h2>

<a href="https://www.freeiconspng.com/img/27213" title="Image from freeiconspng.com"><img src="https://www.freeiconspng.com/uploads/stop-sign-png-8.png" width="350" alt="Png Clipart Collection Stop Sign" /></a>

<p style="color:#2F4F4F;font-size:20px"> Below are some reasons as why you should stop here:</p>
<ul>
    <li style="font-size:18px;color:#2F4F4F;margin:0 0 10px 0"> A data annotation tool like doccano offers manual data annotation only. To make this process automatic we need to use cloud based NLP API services or paid tool like UbiAI, label studio, prodigy etc. I tried UbiAI, label studio both are good but not a perfect one.</li>
    <li style="font-size:18px;color:#2F4F4F;margin:0 0 10px 0"> Do we have existing ML/DL model? If yes then, we need to fine tune our pre-processing as per the existing pipeline. If we don't have any existing pipeline for NLP tasks then need to think about complete architecture.</li>
    <li style="font-size:18px;color:#2F4F4F;margin:0 0 10px 0"> The reason I'm submitting this assignment too late because, "Don't join a company to get your next paycheck." I've explain this in the video (Why should we hire you?) hint: Product Management + Data Analysis (ML) = this combination has tremendous potential to deliver a habit forming product (no matter whether it's B2B or B2C segment)</li>    
</ul>
<h1 style="color:#ff6871;text-align:center"> Thank You!</h1>